In [1]:
import allel

In [2]:
%run workflow/scripts/tools.py
import matplotlib
matplotlib.use('Tkagg')
import matplotlib.pyplot as plt

In [3]:
chroms = ['2L', '2R', '3R', '3L', 'X']

In [4]:
idx = pd.read_csv("resources/reference/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP4.fa.fai", sep="\t", header=None)

In [5]:
aims = zarr.open("resources/gamb_vs_colu.zarr/", mode='r')

In [8]:
### AIMS ###
metadata = pd.read_csv("config/samples.tsv", sep="\t")
metadata = metadata.sort_values(by='species').reset_index(drop=True)
numbers = get_numbers_dict(10)
ploidy = 10
qualflt = 30
missingprop = 0.8

# read AIMs
aims = zarr.open("resources/gamb_vs_colu.zarr/", mode='r')

## initialize dicts
aims_chrom_gamb = {}
aims_chrom_colu = {}
all_gamb = defaultdict(list)
all_colu = defaultdict(list)
n_aims_per_chrom = {}

for chrom in chroms[:1]:

    # read in and filter data
    path = f"results/variants/vcfs/annot.variants.{chrom}.vcf.gz"
    vcf, geno, acsubpops, pos, depth, snpeff, subpops, pops =  readAndFilterVcf(path=path,
                                                               chrom=chrom,
                                                               samples=metadata,
                                                               numbers=numbers,
                                                               ploidy=ploidy,
                                                               qualflt=qualflt,
                                                               missingfltprop=missingprop)
    aimspos = aims[chrom]['POS'][:]

    # get intersection of aims and our SNPs
    aims_pos_mask, aims_mask_2 = pos.locate_intersection(aimspos)
    our_aims = pos[aims_pos_mask]
    print(f"\n In the data, across all samples there are {our_aims.shape[0]} Ancestry Informative markers on Chromosome {chrom}")

    # get gamb and colu alleles, and subset to aims that we have in the rna-seq data 
    aimscolu = aims[chrom]['colu_allele'][:][aims_mask_2]
    aimsgamb = aims[chrom]['gamb_allele'][:][aims_mask_2]

    # get mask that was used in readAndFilterVcf()
    mask = pos.locate_intersection(vcf['variants/POS'])[1]
    ref  = vcf['variants/REF'][mask][aims_pos_mask]
    alt = vcf['variants/ALT'][mask][aims_pos_mask]

    # filter geno array to set of aims
    geno_aims = geno.compress(aims_pos_mask, axis=0)

    totalgambscore = {}
    totalcoluscore = {}

    for aim in our_aims:

        gambscore = {}
        coluscore = {}

        # filter arrays 
        mask = our_aims == aim
        ref_ = ref[mask]
        alt_ = alt[mask]
        aimscolu_ = aimscolu[mask]
        aimsgamb_ = aimsgamb[mask]

        gn_aim = geno_aims.compress(mask, axis=0)

        # convert genotypes to nucleotides
        gn2nucleotide = {0:ref_[0],
                        1:alt_[0][0],
                         2:alt_[0][1],
                         3:alt_[0][2],
                        -1:float("nan")}
        gn = replace_with_dict2_generic(gn_aim, gn2nucleotide)

        # for each sample, get proportion of gambiae/coluzzii alleles
        # alleles that are different to both will be missed here
        for sample in metadata.treatment.unique():
            alleles = gn.take(subpops[sample], axis=1).flatten()
            
            # at each AIM, do we have gamb or colu alleles
            gamb = alleles[alleles != 'nan'] == aimsgamb_
            colu = alleles[alleles != 'nan'] == aimscolu_

            # get proportion of gamb v colu alleles at each locus
            gambscore[sample] = np.mean(gamb)
            coluscore[sample] = np.mean(colu)

        totalgambscore[aim] = dict(gambscore)
        totalcoluscore[aim] = dict(coluscore)
        gambscores = flip_dict(totalgambscore)
        coluscores = flip_dict(totalcoluscore)

        prop_gambiae = {}
        prop_colu = {}
        n_aims_per_sample = {}

        for sample in metadata.treatment.unique():

            prop_gambiae[sample] = np.nanmean(np.array(list(gambscores[sample].values())))
            all_gamb[sample].append(np.nanmean(np.array(list(gambscores[sample].values()))))
            prop_colu[sample] = np.nanmean(np.array(list(coluscores[sample].values())))
            all_colu[sample].append(np.nanmean(np.array(list(coluscores[sample].values()))))
            
            arr = np.array(list(gambscores[sample].values()))
            dim = arr.shape[0]
            n_aims_per_sample[sample] = dim-np.sum(np.isnan(arr))
            
    # store AIM fractions for each chromosome in outer dict 
    aims_chrom_gamb[chrom] = dict(prop_gambiae)
    aims_chrom_colu[chrom] = dict(prop_colu)
    n_aims_per_chrom[chrom] = dict(n_aims_per_sample)


-------------- Reading VCF for chromosome 2L --------------
------- Filtering VCF at QUAL=30 and missingness proportion of 0.8 -------
After QUAL filter, 370890 SNPs retained out of 439212 for chromosome 2L
After missingness filter, 230069 SNPs retained out of 370890 for chromosome 2L

 In the data, across all samples there are 36 Ancestry Informative markers on Chromosome 2L


In [9]:
colu = pd.DataFrame.from_dict(coluscores)
aimscolourdf = pd.merge(aimsposdf, colu, how="outer", left_index=True, right_index=True)
aimscolourdf = aimscolourdf.rename(columns={0:"aimspos"})

aimcolourdf = aimscolourdf.melt(id_vars="aimspos")

for chrom in chroms[:1]:
    #print(chrom)
    length = idx[idx[0] == chrom][1]
    chromaims = aims[f'{chrom}/POS'][:]
    
   # print(length, chromaims)
    plt.figure()
    g=sns.scatterplot(data=aimcolourdf, x='aimspos', y='value', hue="variable", alpha=0.5)
    g.set(ylim=(0, 1))
    g.set_xlim(0, length.iloc[0])
    plt.title(f"AIMs {chrom}")
    plt.show()
    #plt.savefig(f"results/AIMs_arab_{chrom}.locs.png")

NameError: name 'aimsposdf' is not defined

In [10]:
aims = zarr.open("resources/gamb_vs_colu.zarr/", mode='r')


In [17]:
aims.tree()

/
 ├── 2L
 │   ├── POS (95,) int64
 │   ├── colu_allele (95,) object
 │   └── gamb_allele (95,) object
 ├── 2R
 │   ├── POS (58,) int64
 │   ├── colu_allele (58,) object
 │   └── gamb_allele (58,) object
 ├── 3L
 │   ├── POS (30,) int64
 │   ├── colu_allele (30,) object
 │   └── gamb_allele (30,) object
 ├── 3R
 │   ├── POS (57,) int64
 │   ├── colu_allele (57,) object
 │   └── gamb_allele (57,) object
 └── X
     ├── POS (489,) int64
     ├── colu_allele (489,) object
     └── gamb_allele (489,) object

In [25]:
s = {}
for chrom in chroms:
    
    s[chrom] = pd.DataFrame({"chrom" : chrom, 
                  "position": aims[chrom]['POS'][:]})
    
    

In [33]:
pd.concat(s).reset_index(drop=True).to_csv("AIMs_positions.tsv", sep="\t")